L'industrie cinématographique est une industrie de plusieurs milliards de dollars dont les studios s'attendent à faire des bénéfices sur les films qu'ils produisent. Certains d'entre eux cherchent également à satisfaire les critiques de film car une bonne critique peut mener à diverses récompenses et donc une amélioration des recettes. Il est donc nécéssaire pour les producteurs d'être en capacité de prédire, grâce aux différentes informations à leur disposition, la recette de leur futur production ainsi que la note que les critiques peuvent données à leur film. Ainsi, à l'aide des données recueillies dans la base de données IMDB, j'espère pouvoir prédire les recettes mondiale d'un film en utilisant les caractéristiques de films déjà enregistrés ainsi que la note des critiques. 

Afin de comprendre toute la démarche entreprise, le notebook suivra le plan suivant: 

1. Introduction
    1. Scrapping
    2. Nettoyage des données
2. Prédiction du Box office mondial
    1. Prédiction sans les réalisateurs
    2. Prédiction avec les réalisateurs
3. Prédiction de la note Metascore
4. Conclusion



In [1]:
import pandas as pd

In [2]:
import Nettoyage_base_IMDB as nl
import Fonction_notebook_IMDB as fn

In [3]:
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# Introduction
## Scrapping
Par une métode de scrapping via selenium, un total de 7500 films sortis entre 2000 et 2019 ont été extrait avec différentes variables comme le genre, le réalisateur, la langue, le budget, les différentes recettes ou le pays du film. Cette base de données est contenu dans [film_imdb_final.csv](https://github.com/LBoisse/IMDB.git) (le code du scrapping est également disponible dans ce lien). Une première sélection de films a été faite avant le scrapping, ainsi seuls les film sortis au cinéma ont été séléctionnés.

## Nettoyage des données

In [4]:
donnees_brutes = pd.read_csv("film_imdb_final.csv", sep=";")

In [5]:
donnees_nettoyees = (
    donnees_brutes
    .pipe(nl.recode_decimal_millier)  
    .pipe(nl.Recode_Certification)
    .pipe(nl.supression_na)
    .pipe(nl.remplace_na_USA)
    .pipe(nl.recode_genre)
    .pipe(nl.suppression_colonnes)
    .pipe(nl.conversion_type)
)

In [6]:
donnees_nettoyees.head()

,Budget,Semaine 1,Box Mondial,Box USA,Duree,Pays,Langues,Annee,Certification,Metascore,...,Biography,Comedy,Animation,Family,Romance,Music,History,Sport,Horror,Musical
0,185000000,158411483,1005973645,534858444,152,USA,English,2008,G,84,...,0,0,0,0,0,0,0,0,0,0
1,160000000,62785337,836836967,292576195,148,USA,English,2010,G,74,...,0,0,0,0,0,0,0,0,0,0
2,93000000,47211490,888159092,315544750,178,New Zealand,English,2001,G,92,...,0,0,0,0,0,0,0,0,0,0
3,94000000,72629713,1142456987,377845905,201,New Zealand,English,2003,G,94,...,0,0,0,0,0,0,0,0,0,0
4,250000000,160887295,1081141435,448139099,164,UK,English,2012,G,78,...,0,0,0,0,0,0,0,0,0,0


In [7]:
donnees_nettoyees.corr()

,Budget,Semaine 1,Box Mondial,Box USA,Duree,Annee,Metascore,Note IMDB,Vote IMDB,Nombre critiques,...,Biography,Comedy,Animation,Family,Romance,Music,History,Sport,Horror,Musical
Budget,1.000000,0.729118,0.761225,0.720941,0.208920,0.063165,0.037524,0.105063,0.499538,0.453875,...,-0.088017,-0.054868,0.238404,0.255679,-0.142657,-0.063944,0.004503,-0.014346,-0.163394,0.075409
Semaine 1,0.729118,1.000000,0.888888,0.926946,0.153346,0.104967,0.081909,0.121990,0.569059,0.483196,...,-0.111066,-0.021267,0.159747,0.170673,-0.114208,-0.034695,-0.075288,-0.021393,-0.043389,0.044551
Box Mondial,0.761225,0.888888,1.000000,0.938856,0.184042,0.135948,0.177982,0.229086,0.640676,0.528074,...,-0.079844,-0.033045,0.221332,0.213291,-0.106325,-0.032469,-0.045453,-0.041992,-0.089906,0.093454
Box USA,0.720941,0.926946,0.938856,1.000000,0.179399,0.076568,0.185173,0.225897,0.659896,0.518849,...,-0.072555,0.014465,0.209819,0.219139,-0.098362,-0.024274,-0.054458,-0.015701,-0.093919,0.092121
Duree,0.208920,0.153346,0.184042,0.179399,1.000000,0.034049,0.177597,0.268820,0.239503,0.204289,...,0.135048,-0.190315,-0.134694,-0.114420,-0.012332,0.008852,0.120221,0.089770,-0.115320,0.003431
Annee,0.063165,0.104967,0.135948,0.076568,0.034049,1.000000,0.071172,0.032496,0.009804,0.344912,...,0.101745,-0.075718,0.034032,0.004309,-0.118146,-0.022440,0.051145,-0.046882,0.037015,0.003144
Metascore,0.037524,0.081909,0.177982,0.185173,0.177597,0.071172,1.000000,0.738576,0.363166,0.457365,...,0.153560,-0.092513,0.089037,0.008300,-0.010873,0.041123,0.108772,0.007251,-0.154401,0.033773
Note IMDB,0.105063,0.121990,0.229086,0.225897,0.268820,0.032496,0.738576,1.000000,0.475295,0.426203,...,0.192277,-0.170381,0.053328,-0.068290,-0.015999,0.016756,0.128938,0.027226,-0.230858,0.018755
Vote IMDB,0.499538,0.569059,0.640676,0.659896,0.239503,0.009804,0.363166,0.475295,1.000000,0.659713,...,-0.025277,-0.130252,0.022143,-0.025031,-0.113783,-0.044312,-0.026998,-0.043407,-0.079450,-0.007638
Nombre critiques,0.453875,0.483196,0.528074,0.518849,0.204289,0.344912,0.457365,0.426203,0.659713,1.000000,...,0.032401,-0.203552,-0.017675,-0.072896,-0.156710,-0.053691,0.015283,-0.061813,0.058750,0.005503


A la suite de ce nettoyage, on ne conserve que la moitié des films. Tous les films n'ayant pas de résultats aux box office mondial ainsi que de Metascore sont éliminés. Par conséquent, seul 3777 films sont conservés.

On peut également noter, grâce à la matrice de corrélation, que les succés au box office US et le box office à la première semaine sont corrélés avec le Box office mondiale. Ceci est normal car la majorité des films sortent en même temps dans le monde et aux US, et le marché américain n'est pas très différents du marché mondial. 

Afin d'avoir les prédictions les plus logiques possibles, ces deux variables (BO US et Semaine 1) ne seront pas prise en comptes. Ceci est de plus parfaitement logique car les producteurs ne peuvent pas utiliser le box office US pour prédire le Box office Mondial si les films sortent en même temps pour tous les pays du monde.  

# Prédiction du Box office mondial

## Prédiction sans les réalisateurs

In [8]:
variables_quali = ["Pays", "Langues", "Certification"]
variables_quanti = ["Budget","Duree", "Annee", "Metascore",
                    "Action","Crime","Drama","Thriller","Adventure",
                    "Sci-Fi","Fantasy","Western","War","Mystery",
                    "Biography","Comedy","Animation","Family","Romance",
                    "Music","History","Sport","Horror","Musical"]
variable_interet = "Box Mondial"

In [9]:
X_tr, X_te, y_tr, y_te = fn.initialisation_pour_le_traitement(donnees_nettoyees,variables_quali,
                                                              variables_quanti,variable_interet)

In [10]:
modeles_sans_real = fn.Modeles_box_office()

In [11]:
fn.affichage_meilleurs_modeles_entrainement(modeles_sans_real, X_tr, y_tr)

RandomForestRegressor(n_estimators=300)                                0.642   0.080838
RandomForestRegressor(n_estimators=350)                                0.639   0.079802
RandomForestRegressor(n_estimators=400)                                0.638   0.082578
RandomForestRegressor(n_estimators=200)                                0.638   0.084874
RandomForestRegressor(n_estimators=150)                                0.637   0.086752
RandomForestRegressor(n_estimators=250)                                0.636   0.082099
RandomForestRegressor()                                                0.636   0.090839
RandomForestRegressor(n_estimators=50)                                 0.633   0.08825
SVR(C=1000000000, epsilon=1)                                           0.624   0.057805
SVR(C=1000000000)                                                      0.624   0.057805
SVR(C=1000000000, epsilon=0.01)                                        0.624   0.057805
SVR(C=1000000000, epsilon=0.001) 

Les résultats des différents modèles montrent que la Random Forest à 300 n_estimators est le meilleur modèle de prédiction sur les données d'entraitement. Les SVR avec pour paramètre C=1000000000 ont également de très bon résultats et possèdent une variance inférieur aux Random Forest. Enfin, les modèles Ridge ont un score qui semblait augmenter avec alpha. Cependant on voit bien que le maximum de ces modèles est atteint pour alpha = 1. On pourrait donc décider de prendre le Random Forest à 300 n_estimators pour modèle, mais on va d'abort essayer des modèles qui prennent en compte les réalisateurs.

## Prédiction avec les réalisateurs

On réalise donc la même démarche que précédemment mais on ajoute les réalisateurs dans la variables expliquatives.

In [15]:
variables_quali = [ "Pays", "Langues", "Certification", "Realisateurs"]
variables_quanti = ["Budget","Duree", "Annee", "Metascore",
                    "Action","Crime","Drama","Thriller","Adventure",
                    "Sci-Fi","Fantasy","Western","War","Mystery",
                    "Biography","Comedy","Animation","Family","Romance",
                    "Music","History","Sport","Horror","Musical"]
variable_interet = "Box Mondial"

In [16]:
X_tr, X_te, y_tr, y_te = fn.initialisation_pour_le_traitement(donnees_nettoyees,variables_quali,
                                                              variables_quanti, variable_interet)

In [17]:
modeles_avec_real = fn.Modeles_box_office()

In [15]:
fn.affichage_meilleurs_modeles_entrainement(modeles_avec_real, X_tr, y_tr)

RandomForestRegressor(n_estimators=50)                                 0.656   0.083161
RandomForestRegressor(n_estimators=250)                                0.656   0.080932
RandomForestRegressor(n_estimators=300)                                0.655   0.08072
RandomForestRegressor(n_estimators=350)                                0.654   0.081951
RandomForestRegressor(n_estimators=400)                                0.653   0.079352
RandomForestRegressor(n_estimators=200)                                0.653   0.078961
RandomForestRegressor()                                                0.653   0.080308
RandomForestRegressor(n_estimators=150)                                0.65   0.082915
SVR(C=1000000000, epsilon=1)                                           0.621   0.045143
SVR(C=1000000000)                                                      0.621   0.045143
SVR(C=1000000000, epsilon=0.01)                                        0.621   0.045143
SVR(C=1000000000, epsilon=0.001)  

L'ajout des réalisateurs améliore très légérement les résultats sur les données d'entrainement. Ce sont toujours les Random Forest qui donnent les meilleurs score mais contrairement au résultat sans les réal, c'est la Random Forest à 50 n_estimator qui donne le plus de satisfaction. Les SVR à C=1000000000 donnent toujours de bon résultats avec un variance inférieur aux Random Forest. Afin de choisir entre les Random Forest avec ou sans réalisateurs, le temps de calcul peut être pris en compte étant donnée que les résultats sont assez similaires. Cependant, le temps de calcul ici en prenant en compte les réalisateurs n'est pas excessif, le modèle chois est donc la Random Forest à 50 n_estimators avec les réalisateurs. 

In [21]:
fn.score_modele_data_test(RandomForestRegressor(n_estimators=50), X_tr, y_tr, X_te, y_te )

Score données entrainement : 0.654
Score données test : 0.634


Les résultats montrent que sur les données test, le modèle choisis est valide dans 63% des cas. On peut observer qu'il n'y a pas eu trop de surapprentissage sur les données d'entrainement. Il est assez compliqué de jugé un film sur ses caractéristiques seulement donc les résultats obtenue sont bon.

# Prédiction de la note Metascore

Tout aussi important pour les producteurs, la note des critiques doit être prédite. On réalise donc la même démarche en changeant la variables d'intérêt.

In [22]:
variables_quali = [ "Pays", "Langues", "Certification", "Realisateurs"]
variables_quanti = ["Budget", "Duree", "Annee", "Nombre critiques", "Semaine 1",
                    "Crime","Drama","Thriller","Adventure","Sci-Fi","Fantasy",
                    "Western","War","Mystery","Biography","Comedy","Animation",
                    "Family","Romance","Music","History","Sport","Horror","Musical", "Action"]
variable_interet = "Metascore"

In [23]:
X_tr, X_te, y_tr, y_te = fn.initialisation_pour_le_traitement(donnees_nettoyees,variables_quali,
                                                              variables_quanti, variable_interet)

In [24]:
modele_metascore = fn.Modeles_note_Metascore()

In [19]:
fn.affichage_meilleurs_modeles_entrainement(modele_metascore, X_tr, y_tr)

RandomForestRegressor(n_estimators=150)                                0.551   0.045246
RandomForestRegressor(n_estimators=400)                                0.55   0.047005
RandomForestRegressor(n_estimators=350)                                0.55   0.04572
RandomForestRegressor(n_estimators=300)                                0.55   0.045672
RandomForestRegressor(n_estimators=250)                                0.55   0.046169
RandomForestRegressor(n_estimators=200)                                0.549   0.045777
RandomForestRegressor()                                                0.548   0.05045
RandomForestRegressor(n_estimators=50)                                 0.544   0.048058
SVR(C=100, epsilon=1)                                                  0.488   0.029759
SVR(C=100)                                                             0.484   0.030538
SVR(C=100, epsilon=0.01)                                               0.484   0.030575
SVR(C=100, epsilon=0.001)             

Les Random Forest, et plus précisement celle à 150 n_estimator, donnent les meilleurs score de prédiction sur les données d'entrainement. La variance reste assez faible de plus donc le modèle choisis est la Random Forest à 150 n_estimators

In [26]:
fn.score_modele_data_test(RandomForestRegressor(n_estimators=150), X_tr, y_tr, X_te, y_te )

Score données entrainement : 0.551
Score données test : 0.535


Les résultats montrent que sur les données test, le modèle choisis est valide dans 54% des cas. On peut observer qu'il n'y a pas eu trop de surapprentissage sur les données d'entrainement. Il est assez compliqué de jugé un film sur ses caractéristiques seulement donc les résultats obtenue sont bon.

# Conclusion

Il est assez difficile de prédire les résultats pour un film car l'attrait du publique et des critiques pour ceux ci ne dépendent pas forcément que de variables techniques. En effet, il y a une part difficilement observable qui font le succés d'un film.

# Axe de recherche pour la suite

- Ajouter une popularité aux réalisateurs au moment de la sortie du film 
- Faire de même pour les acteurs. On pourrait prendre les 3,4 ou 5 acteurs principaux 
- Afin de prendre en compte les notes IMDB, on peut voir s'il est possible de récuperer les notes moyenne ou médianne après un, deux , trois jours ou bien après une semaine d'exploitation du film aux USA pour en faire des variables

Toutes ces données sont disponibles (du moins pour les deux premières) sur IMDB pro. Nécéssite un abonnement payant (essaie gratuit si j'ai un mois libre)